In [1]:
%pip install langgraph


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 KB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 KB 3.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 KB 5.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 KB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 5.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.128
    Uninstalling langsmith-0.1.128:
      Successfully uninstalled langsmith-0.1.128
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.31
    Uninstalling langchain-core-0.3.31:
      Succ

In [ ]:
import pandas as pd
import numpy as np
from typing import TypedDict, Annotated, List, Dict, Any
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
import operator
import random
from dotenv import load_dotenv
import os
import google.generativeai as genai

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

print("🎭 BUILDING THERAPEUTIC CHATBOT WITH LANGGRAPH + GEMINI")
print("Multi-agent system with 3 therapeutic personas")
print("="*60)

# 1. Create simple Gemini wrapper (no inheritance from BaseLanguageModel)
class GeminiLLM:
    """Simple Gemini LLM wrapper"""
    
    def __init__(self, model_name: str = "gemini-1.5-flash"):
        self.model = genai.GenerativeModel(model_name)
    
    def invoke(self, messages: List[BaseMessage], **kwargs) -> AIMessage:
        """Invoke method for compatibility with LangChain"""
        
        # Convert LangChain messages to a single prompt
        if len(messages) == 1 and isinstance(messages[0], SystemMessage):
            prompt = messages[0].content
        else:
            # Combine messages into a single prompt
            prompt_parts = []
            for msg in messages:
                if isinstance(msg, SystemMessage):
                    prompt_parts.append(f"System: {msg.content}")
                elif isinstance(msg, HumanMessage):
                    prompt_parts.append(f"Human: {msg.content}")
                elif isinstance(msg, AIMessage):
                    prompt_parts.append(f"Assistant: {msg.content}")
                else:
                    prompt_parts.append(str(msg.content))
            prompt = "\n".join(prompt_parts)
        
        try:
            # Generate response using Gemini
            response = self.model.generate_content(prompt)
            
            # Extract text from response
            if response.text:
                text = response.text
            else:
                text = "I apologize, but I couldn't generate a response. Please try again."
            
            return AIMessage(content=text)
            
        except Exception as e:
            print(f"❌ Gemini API Error: {e}")
            fallback_text = "I'm having trouble connecting right now. Please try again."
            return AIMessage(content=fallback_text)

# Initialize Gemini LLM
print("🔧 Initializing Gemini LLM...")
llm = GeminiLLM("gemini-1.5-flash")
print("✅ Gemini LLM ready!")

# 2. Load our semantic personas
print("📁 Loading therapeutic personas...")

therapist_personas = pd.read_csv('semantic_therapist_personas.csv')
wise_personas = pd.read_csv('semantic_wise_personas.csv')
intelligent_personas = pd.read_csv('semantic_intelligent_personas.csv')

print(f"✅ Loaded personas:")
print(f"   😊 Therapist: {len(therapist_personas)} characters")
print(f"   🧠 Wise Mentor: {len(wise_personas)} characters") 
print(f"   🎓 Intelligent Expert: {len(intelligent_personas)} characters")

# 3. Define the shared state for our multi-agent system
class ChatState(TypedDict):
    messages: Annotated[List[dict], operator.add]
    user_input: str
    user_emotion: str
    conversation_context: str
    selected_persona: str
    response_history: List[dict]
    session_id: str

# 4. Create knowledge bases for each persona
class PersonaKnowledge:
    def __init__(self, persona_df: pd.DataFrame, persona_type: str):
        self.persona_type = persona_type
        self.characters = persona_df
        self.knowledge_base = self._create_knowledge_base()
    
    def _create_knowledge_base(self) -> Dict[str, Any]:
        """Create a knowledge base from character dialogues"""
        knowledge = {
            'character_examples': [],
            'communication_patterns': [],
            'key_phrases': [],
            'behavioral_traits': []
        }
        
        # Extract top 5 characters for this persona
        top_characters = self.characters.head(5)
        
        for _, char in top_characters.iterrows():
            char_info = {
                'name': char['character_name'],
                'movie': char['movie_title'],
                'dialogue_sample': char['dialogue'],
                'sentiment_profile': {
                    'polarity': char['polarity'],
                    'subjectivity': char['subjectivity'],
                    'intensity': char['intensity']
                }
            }
            knowledge['character_examples'].append(char_info)
        
        return knowledge
    
    def get_character_inspiration(self) -> dict:
        """Get a random character for inspiration"""
        char = self.characters.sample(1).iloc[0]
        return {
            'name': char['character_name'],
            'dialogue': char['dialogue'],
            'movie': char['movie_title']
        }

# Initialize persona knowledge bases
print("\n🧠 Creating persona knowledge bases...")

therapist_kb = PersonaKnowledge(therapist_personas, "Optimistic Therapist")
wise_kb = PersonaKnowledge(wise_personas, "Wise Mentor")
intelligent_kb = PersonaKnowledge(intelligent_personas, "Intelligent Expert")

print("✅ Knowledge bases created!")

# 5. Create agent prompts
def create_therapist_prompt(character_inspiration: str, user_input: str) -> str:
    return f"""You are an OPTIMISTIC THERAPIST agent inspired by characters like:
- DR. TAYLOR (A Clockwork Orange): "I'm sure it won't be long now... you seem well on the way to complete recovery"
- LEADER (Fight Club): "Tell the other person how you feel. Find your power animal."

Your role:
- Provide emotional support and encouragement
- Ask therapeutic questions to help users explore their feelings
- Validate emotions and offer hope
- Use gentle, empathetic language
- Focus on the user's strengths and potential for growth

Character inspiration: {character_inspiration}

User says: "{user_input}"

Respond in a supportive, therapeutic manner. Keep responses concise but meaningful (2-3 sentences max)."""

def create_wise_mentor_prompt(character_inspiration: str, user_input: str) -> str:
    return f"""You are a WISE MENTOR agent inspired by characters like:
- KRINGELEIN (Grand Hotel): "Every glass high to life... the courage to live it"
- SOMERSET (Seven): Thoughtful, reflective wisdom
- MIRACLE MAX (Princess Bride): "True love is the greatest thing in the world"

Your role:
- Share life wisdom and perspective
- Offer guidance based on experience
- Use metaphors and stories when appropriate
- Speak with calm authority and patience
- Help users see the bigger picture

Character inspiration: {character_inspiration}

User says: "{user_input}"

Respond with wisdom and life perspective. Draw from experience and deeper truths (2-3 sentences max)."""

def create_intelligent_expert_prompt(character_inspiration: str, user_input: str) -> str:
    return f"""You are an INTELLIGENT EXPERT agent inspired by characters like:
- SPOCK (Star Trek): "I'm not human... logically..."
- DEAN FULTON (Time Machine): "Abstract mathematics, relativity of dimension"
- OCK (Spider-Man): "This is the unified field! All forces of the universe!"

Your role:
- Provide analytical and logical perspectives
- Break down complex problems systematically
- Offer fact-based insights and solutions
- Use precise, clear language
- Help users think through problems rationally

Character inspiration: {character_inspiration}

User says: "{user_input}"

Respond with intelligence and analytical thinking. Focus on logic and systematic problem-solving (2-3 sentences max)."""

# 6. Define agent functions
def therapist_agent(state: ChatState) -> ChatState:
    """Optimistic Therapist Agent"""
    print("😊 Therapist agent responding...")
    
    # Get character inspiration
    inspiration = therapist_kb.get_character_inspiration()
    
    # Create prompt
    prompt_text = create_therapist_prompt(
        f"Channel {inspiration['name']} from {inspiration['movie']}",
        state['user_input']
    )
    
    # Generate response
    response = llm.invoke([SystemMessage(content=prompt_text)])
    
    # Update state
    new_message = {
        'role': 'therapist',
        'content': response.content,
        'inspiration': inspiration['name']
    }
    
    return {
        **state,
        'messages': [new_message],
        'selected_persona': 'therapist'
    }

def wise_mentor_agent(state: ChatState) -> ChatState:
    """Wise Mentor Agent"""
    print("🧠 Wise Mentor agent responding...")
    
    # Get character inspiration
    inspiration = wise_kb.get_character_inspiration()
    
    # Create prompt
    prompt_text = create_wise_mentor_prompt(
        f"Channel {inspiration['name']} from {inspiration['movie']}",
        state['user_input']
    )
    
    # Generate response
    response = llm.invoke([SystemMessage(content=prompt_text)])
    
    # Update state
    new_message = {
        'role': 'wise_mentor',
        'content': response.content,
        'inspiration': inspiration['name']
    }
    
    return {
        **state,
        'messages': [new_message],
        'selected_persona': 'wise_mentor'
    }

def intelligent_expert_agent(state: ChatState) -> ChatState:
    """Intelligent Expert Agent"""
    print("🎓 Intelligent Expert agent responding...")
    
    # Get character inspiration
    inspiration = intelligent_kb.get_character_inspiration()
    
    # Create prompt
    prompt_text = create_intelligent_expert_prompt(
        f"Channel {inspiration['name']} from {inspiration['movie']}",
        state['user_input']
    )
    
    # Generate response
    response = llm.invoke([SystemMessage(content=prompt_text)])
    
    # Update state
    new_message = {
        'role': 'intelligent_expert',
        'content': response.content,
        'inspiration': inspiration['name']
    }
    
    return {
        **state,
        'messages': [new_message],
        'selected_persona': 'intelligent_expert'
    }

# 7. Router function to select appropriate agent
def route_to_agent(state: ChatState) -> str:
    """Route user input to the most appropriate agent"""
    user_input = state['user_input'].lower()
    
    # Enhanced keyword-based routing
    therapy_keywords = ['feel', 'feeling', 'sad', 'depressed', 'anxious', 'worried', 'stressed', 'emotional', 'upset', 'hurt', 'angry', 'scared', 'lonely', 'overwhelmed']
    wisdom_keywords = ['advice', 'guidance', 'life', 'meaning', 'purpose', 'direction', 'confused', 'lost', 'stuck', 'crossroads', 'decision', 'choice']
    expert_keywords = ['problem', 'solve', 'analyze', 'think', 'logic', 'rational', 'plan', 'strategy', 'solution', 'approach', 'method', 'calculate']
    
    therapy_score = sum(1 for word in therapy_keywords if word in user_input)
    wisdom_score = sum(1 for word in wisdom_keywords if word in user_input)
    expert_score = sum(1 for word in expert_keywords if word in user_input)
    
    print(f"🎯 Routing scores - Therapy: {therapy_score}, Wisdom: {wisdom_score}, Expert: {expert_score}")
    
    if therapy_score >= wisdom_score and therapy_score >= expert_score:
        return "therapist"
    elif wisdom_score >= expert_score:
        return "wise_mentor"
    else:
        return "intelligent_expert"

# 8. Create the LangGraph workflow
print("\n🔧 Building LangGraph workflow...")

# Initialize the graph
workflow = StateGraph(ChatState)

# Add nodes for each agent
workflow.add_node("therapist", therapist_agent)
workflow.add_node("wise_mentor", wise_mentor_agent)
workflow.add_node("intelligent_expert", intelligent_expert_agent)

# Set entry point based on routing
workflow.set_conditional_entry_point(
    route_to_agent,
    {
        "therapist": "therapist",
        "wise_mentor": "wise_mentor", 
        "intelligent_expert": "intelligent_expert"
    }
)

# Compile the graph
app = workflow.compile()

print("✅ LangGraph workflow created!")

# 9. Demo function
def run_demo():
    """Run a demo of the therapeutic chatbot"""
    print("\n🎬 RUNNING DEMO...")
    print("Testing all 3 therapeutic personas with different types of input")
    
    demo_inputs = [
        "I'm feeling really anxious about my job interview tomorrow",
        "I don't know what direction my life is going in", 
        "I need help solving a complex problem at work"
    ]
    
    for i, demo_input in enumerate(demo_inputs, 1):
        print(f"\n{'='*60}")
        print(f"Demo {i}/3")
        print(f"{'='*60}")
        print(f"You: {demo_input}")
        
        initial_state = {
            'messages': [],
            'user_input': demo_input,
            'user_emotion': '',
            'conversation_context': '',
            'selected_persona': '',
            'response_history': [],
            'session_id': f"demo_{i}"
        }
        
        try:
            result = app.invoke(initial_state)
            
            if result['messages']:
                message = result['messages'][-1]
                persona_emoji = {
                    'therapist': '😊',
                    'wise_mentor': '🧠',
                    'intelligent_expert': '🎓'
                }
                
                emoji = persona_emoji.get(message['role'], '🤖')
                persona_name = message['role'].replace('_', ' ').title()
                
                print(f"\n{emoji} {persona_name} (inspired by {message['inspiration']}):")
                print(f"{message['content']}")
                
        except Exception as e:
            print(f"❌ Error in demo: {e}")
            import traceback
            traceback.print_exc()

# 10. Interactive chat function
def chat_with_therapeutic_bot():
    """Interactive chat interface"""
    print(f"\n{'='*60}")
    print("🎭 THERAPEUTIC CHATBOT READY!")
    print("Your AI therapeutic companions will automatically select based on your needs:")
    print("😊 Therapist - For emotional support and feelings")
    print("🧠 Wise Mentor - For life guidance and direction") 
    print("🎓 Intelligent Expert - For problem solving and analysis")
    print("\nType 'quit' to exit")
    print(f"{'='*60}")
    
    session_id = f"session_{random.randint(1000, 9999)}"
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'bye', 'goodbye']:
            print("\n👋 Thank you for chatting! Take care of yourself!")
            break
            
        if not user_input:
            print("Please enter something to get started...")
            continue
            
        # Create initial state
        initial_state = {
            'messages': [],
            'user_input': user_input,
            'user_emotion': '',
            'conversation_context': '',
            'selected_persona': '',
            'response_history': [],
            'session_id': session_id
        }
        
        try:
            # Run the workflow
            result = app.invoke(initial_state)
            
            # Display response
            if result['messages']:
                message = result['messages'][-1]
                persona_emoji = {
                    'therapist': '😊',
                    'wise_mentor': '🧠', 
                    'intelligent_expert': '🎓'
                }
                
                emoji = persona_emoji.get(message['role'], '🤖')
                persona_name = message['role'].replace('_', ' ').title()
                
                print(f"\n{emoji} {persona_name} (inspired by {message['inspiration']}):")
                print(f"{message['content']}")
                print("-" * 50)
            
        except Exception as e:
            print(f"❌ Error: {e}")
            print("🔄 Please try again...")
            
# Simple chat function that works in Jupyter/Colab
def simple_chat(user_message):
    """Chat with one message at a time"""
    
    # Create initial state
    initial_state = {
        'messages': [],
        'user_input': user_message,
        'user_emotion': '',
        'conversation_context': '',
        'selected_persona': '',
        'response_history': [],
        'session_id': 'simple_chat'
    }
    
    print(f"You: {user_message}")
    
    try:
        # Run the workflow
        result = app.invoke(initial_state)
        
        # Display response
        if result['messages']:
            message = result['messages'][-1]
            persona_emoji = {
                'therapist': '😊',
                'wise_mentor': '🧠', 
                'intelligent_expert': '🎓'
            }
            
            emoji = persona_emoji.get(message['role'], '🤖')
            persona_name = message['role'].replace('_', ' ').title()
            
            print(f"\n{emoji} {persona_name} (inspired by {message['inspiration']}):")
            print(f"{message['content']}")
            print("-" * 50)
        
    except Exception as e:
        print(f"❌ Error: {e}")

# Now you can chat like this:
print("🎭 SIMPLE CHAT MODE - Just run the function with your message!")
print("Example: simple_chat('your message here')")
if __name__ == "__main__":
    print("\n🚀 THERAPEUTIC CHATBOT INITIALIZATION COMPLETE!")
    
    # Run demo first
    run_demo()
    
    # Then start interactive chat
    print("\n" + "="*60)
    print("DEMO COMPLETE - Starting Interactive Mode")
    chat_with_therapeutic_bot()

/home/samanvitha/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🎭 BUILDING THERAPEUTIC CHATBOT WITH LANGGRAPH + GEMINI
Multi-agent system with 3 therapeutic personas
🔧 Initializing Gemini LLM...
✅ Gemini LLM ready!
📁 Loading therapeutic personas...
✅ Loaded personas:
   😊 Therapist: 20 characters
   🧠 Wise Mentor: 20 characters
   🎓 Intelligent Expert: 20 characters

🧠 Creating persona knowledge bases...
✅ Knowledge bases created!

🔧 Building LangGraph workflow...
✅ LangGraph workflow created!
🎭 SIMPLE CHAT MODE - Just run the function with your message!
Example: simple_chat('your message here')

🚀 THERAPEUTIC CHATBOT INITIALIZATION COMPLETE!

🎬 RUNNING DEMO...
Testing all 3 therapeutic personas with different types of input

Demo 1/3
You: I'm feeling really anxious about my job interview tomorrow
🎯 Routing scores - Therapy: 3, Wisdom: 0, Expert: 0
😊 Therapist agent responding...

😊 Therapist (inspired by MR. HENRY):
That's completely understandable!  It's natural to feel anxious before a big interview.  Let's explore what's fueling that anxiety, a

In [ ]:
simple_chat("I'm feeling overwhelmed with work")

NameError: name 'simple_chat' is not defined